In [44]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
#from keras.layers import Dense, Conv2D, Deconvolution2D, MaxPooling2D, Flatten
from keras import layers as L
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, SeparableConv2D

In [45]:
pwd

'C:\\Users\\david\\Desktop\\DATAMAD-0819\\laboratorios\\Proyecto_Final\\SRC'

In [46]:
path = '../Euro_Sat/'

In [47]:
dic = {}
entrada = []
salida = []
i = 0

for ele in os.listdir(path):
    if os.path.isdir(path+ele):
        for file in os.listdir(path+ele):
            im = Image.open(os.path.join(path,ele,file))
            im_array = np.array(im)
            entrada.append(im_array)
            salida.append(i)
        dic[i] = ele
        i += 1


In [56]:
#from sklearn import preprocessing

In [60]:
#le = preprocessing.LabelEncoder()
#print(le.fit(["AnnualCrop", "Forest", "HerbaceousVegetation", "Highway", "Industrial",
        "Pasture", "PermanentCrop", "Residential", "river", "seaLake"]))

#list(le.classes_)



#list(le.inverse_transform([2, 2, 1]))


LabelEncoder()


['AnnualCrop',
 'Forest',
 'HerbaceousVegetation',
 'Highway',
 'Industrial',
 'Pasture',
 'PermanentCrop',
 'Residential',
 'river',
 'seaLake']

In [71]:
#print(le.transform(["AnnualCrop", "Forest", "HerbaceousVegetation", "Highway", "Industrial",
        "Pasture", "PermanentCrop", "Residential", "river", "seaLake"]))

[0 1 2 3 4 5 6 7 8 9]


In [66]:
#pru = Image.open("../SRC/test2.jpg")
#pru_array = np.array(im)
#test_image = np.array([pru_array])

In [67]:
#model.predict(test_image)

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

In [48]:
clases_salida = to_categorical(salida)

In [49]:
entrada = np.array(entrada)
entrada = entrada/255

In [50]:
x_train, x_test, y_train, y_test = train_test_split(entrada, clases_salida, test_size=0.2, shuffle=True)

In [51]:
in_shape = x_train[0,:,:,:].shape

model = Sequential()

model.add(Conv2D(16, kernel_size=3, activation='relu', input_shape=in_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(50))
model.add(Dense(len(dic), activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 62, 62, 16)        448       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 31, 31, 16)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 29, 29, 32)        4640      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 12, 12, 64)        18496     
_________________________________________________________________
flatten_4 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 50)               

In [52]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=25)

Train on 21600 samples, validate on 5400 samples
Epoch 1/25
21600/21600 [==============================] - 66s 3ms/step - loss: 1.1531 - accuracy: 0.5756 - val_loss: 0.8646 - val_accuracy: 0.6891
Epoch 2/25
21600/21600 [==============================] - 70s 3ms/step - loss: 0.7352 - accuracy: 0.7356 - val_loss: 0.6992 - val_accuracy: 0.7567
Epoch 3/25
21600/21600 [==============================] - 68s 3ms/step - loss: 0.6384 - accuracy: 0.7710 - val_loss: 0.5653 - val_accuracy: 0.7969
Epoch 4/25
21600/21600 [==============================] - 68s 3ms/step - loss: 0.5457 - accuracy: 0.8030 - val_loss: 0.5906 - val_accuracy: 0.7880
Epoch 5/25
21600/21600 [==============================] - 68s 3ms/step - loss: 0.4821 - accuracy: 0.8270 - val_loss: 0.5450 - val_accuracy: 0.8065
Epoch 6/25
21600/21600 [==============================] - 68s 3ms/step - loss: 0.4347 - accuracy: 0.8438 - val_loss: 0.5214 - val_accuracy: 0.8148
Epoch 7/25
21600/21600 [==============================] - 68s 3ms/ste

In [54]:
scores = model.evaluate(x_train, y_train, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 98.60%


In [55]:
model_json = model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model1.h5")
print("Saved model to disk")

Saved model to disk


In [56]:
prueba_test = model.predict(x_test)

In [66]:
for prueba in prueba_test[19]:
    print(round(prueba,2))

0.0
0.0
0.98
0.0
0.0
0.0
0.0
0.02
0.0
0.0


In [67]:
pred_test = [dic[np.argmax(p)] for p in prueba_test]

In [68]:
suelos_test = [dic[np.argmax(s)] for s in y_test]

In [69]:
s = suelos_test[50:90]

In [70]:
t = pred_test[50:90]

In [76]:
import pandas as pd

d = {'y_Actual': s,
        'y_Predicted': t}

dat = pd.DataFrame(data=d)
dat


,y_Actual,y_Predicted
0,HerbaceousVegetation,HerbaceousVegetation
1,PermanentCrop,River
2,Industrial,Industrial
3,HerbaceousVegetation,HerbaceousVegetation
4,Highway,Highway
5,PermanentCrop,Highway
6,HerbaceousVegetation,HerbaceousVegetation
7,River,River
8,Forest,Forest
9,Forest,Forest


In [65]:
from sklearn.metrics import confusion_matrix
confusion_matrix(s, t)

array([[4, 0, 0, 1, 0, 0, 1, 0, 0, 0],
       [0, 5, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 7, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 5, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 2, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 3, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 4]], dtype=int64)

In [30]:
import matplotlib.pyplot as plt
my_image_resized = plt.imread("../Pruebas/test_sea.jpg")

In [31]:
import numpy as np
probabilities = model.predict(np.array([my_image_resized,]))

In [32]:
probabilities

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

In [33]:
number_to_class = ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial',
                   'Pasture', 'PermanentCrop', 'Residential', 'river', 'seaLake']

In [34]:
index = np.argsort(probabilities[0,:])

In [35]:
print("Most likely class:", number_to_class[index[9]], "-- Probability:", probabilities[0,index[9]])
print("Second most likely class:", number_to_class[index[8]], "-- Probability:", probabilities[0,index[8]])
print("Third most likely class:", number_to_class[index[7]], "-- Probability:", probabilities[0,index[7]])
print("Fourth most likely class:", number_to_class[index[6]], "-- Probability:", probabilities[0,index[6]])
print("Fifth most likely class:", number_to_class[index[5]], "-- Probability:", probabilities[0,index[5]])
print("sixth most likely class:", number_to_class[index[9]], "-- Probability:", probabilities[0,index[4]])
print("seventh most likely class:", number_to_class[index[8]], "-- Probability:", probabilities[0,index[3]])
print("eight most likely class:", number_to_class[index[7]], "-- Probability:", probabilities[0,index[2]])
print("nineth most likely class:", number_to_class[index[6]], "-- Probability:", probabilities[0,index[1]])
print("tenth most likely class:", number_to_class[index[5]], "-- Probability:", probabilities[0,index[0]])

Most likely class: Industrial -- Probability: 1.0
Second most likely class: seaLake -- Probability: 0.0
Third most likely class: river -- Probability: 0.0
Fourth most likely class: Residential -- Probability: 0.0
Fifth most likely class: PermanentCrop -- Probability: 0.0
sixth most likely class: Industrial -- Probability: 0.0
seventh most likely class: seaLake -- Probability: 0.0
eight most likely class: river -- Probability: 0.0
nineth most likely class: Residential -- Probability: 0.0
tenth most likely class: PermanentCrop -- Probability: 0.0


In [25]:
model.predict(x_test[0:2])

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 7.92483687e-26,
        1.00000000e+00, 0.00000000e+00, 4.23117161e-36, 0.00000000e+00,
        8.26051013e-34, 1.50046975e-35],
       [1.68201897e-08, 9.92738114e-10, 1.01622425e-01, 5.49199001e-04,
        3.98225410e-07, 4.24567588e-06, 8.97823274e-01, 2.67768229e-07,
        7.13268236e-08, 3.52467083e-11]], dtype=float32)